In [2]:
import os
import time
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.tools import tool
from together import Together
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import LLMResult, Generation
from typing import Any, List, Optional

# Load environment variables
load_dotenv()

# Initialize Together AI client
client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

class TogetherAILLM(BaseLLM):
    """LangChain LLM wrapper for Together AI"""
    
    model: str = "meta-llama/Llama-3-8b-chat-hf"
    temperature: float = 0.7
    max_tokens: int = 2048
    
    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Any = None,
        **kwargs: Any,
    ) -> LLMResult:
        generations = []
        for prompt in prompts:
            try:
                response = client.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=self.temperature,
                    max_tokens=self.max_tokens,
                    stop=stop,
                    **kwargs
                )
                text = response.choices[0].message.content
                generations.append([Generation(text=text)])
            except Exception as e:
                if run_manager:
                    run_manager.on_llm_error(e)
                raise
        
        return LLMResult(generations=generations)
    
    def _llm_type(self) -> str:
        return "together_ai"
    
    def bind(self, stop: Optional[List[str]] = None, **kwargs: Any) -> BaseLLM:
        return self.copy(update={**kwargs, **({"stop": stop} if stop else {})})

# Initialize LLM
llm = TogetherAILLM()

@tool
def chrome_research(query: str, max_results: int = 3) -> str:
    """Perform web research using Chrome browser."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        driver.get(f"https://www.google.com/search?q={query}")
        time.sleep(2)
        results = []
        links = driver.find_elements(By.CSS_SELECTOR, "div.g a")[:max_results]
        
        for i, link in enumerate(links):
            href = link.get_attribute("href")
            if href and href.startswith("http"):
                driver.execute_script(f"window.open('{href}');")
                driver.switch_to.window(driver.window_handles[i+1])
                time.sleep(2)
                
                try:
                    visible_text = driver.find_element(By.TAG_NAME, "body").text
                    results.append(visible_text[:5000])
                except Exception as e:
                    print(f"Error extracting text from {href}: {str(e)}")
                
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        
        return "\n\n".join(results)
    
    except Exception as e:
        print(f"Error during research: {str(e)}")
        return f"Research failed: {str(e)}"
    finally:
        driver.quit()

def setup_agent():
    """Set up the research agent and executor"""
    tools = [chrome_research]
    research_prompt = hub.pull("hwchase17/react")
    research_agent = create_react_agent(llm, tools, research_prompt)
    return AgentExecutor(agent=research_agent, tools=tools, verbose=True)

def get_user_topic():
    """Get research topic from user with validation"""
    while True:
        topic = input("\nWhat would you like to research? (or 'quit' to exit): ").strip()
        if topic.lower() == 'quit':
            return None
        if len(topic.split()) < 2:
            print("Please enter a more specific topic (at least 2 words).")
            continue
        if len(topic) > 200:
            print("Topic too long. Please keep it under 200 characters.")
            continue
        return topic

def research_workflow(topic: str, research_executor: AgentExecutor):
    """Execute the full research workflow for a given topic"""
    print(f"\nStarting research on: {topic}")
    
    try:
        # Generate research questions
        questions = llm.invoke(f"Generate 3 specific research questions about {topic}")
        questions = [q.strip() for q in questions.split("\n") if q.strip()][:3]
        
        # Conduct research
        findings = []
        for question in questions:
            try:
                print(f"\nResearching: {question}")
                research = research_executor.invoke({"input": question})
                findings.append(research["output"])
            except Exception as e:
                print(f"Error researching {question}: {str(e)}")
                findings.append(f"Failed to research: {question}")
        
        # Generate report
        report = llm.invoke(
            f"Summarize these research findings about {topic}:\n{'\n'.join(findings)}\n\n"
            "Provide a well-structured report with key points and sources."
        )
        
        return {
            "questions": questions,
            "findings": findings,
            "report": report
        }
    except Exception as e:
        print(f"Error in research workflow: {str(e)}")
        return {
            "error": str(e),
            "questions": [],
            "findings": [],
            "report": ""
        }

def display_results(results: dict):
    """Display research results in a user-friendly format"""
    if "error" in results:
        print("\nWorkflow failed with error:", results["error"])
        return
    
    print("\n" + "="*50)
    print(" RESEARCH RESULTS ".center(50, "="))
    print("="*50)
    
    print("\nRESEARCH QUESTIONS:")
    for i, question in enumerate(results["questions"], 1):
        print(f"{i}. {question}")
    
    print("\nFINAL REPORT:")
    print(results["report"])
    print("\n" + "="*50)

def main():
    """Main interactive research loop"""
    print("\n" + "="*50)
    print(" AI RESEARCH ASSISTANT ".center(50, "="))
    print("="*50)
    print("\nI can help research any topic and generate a detailed report.")
    print("Just tell me what you'd like to research!\n")
    
    research_executor = setup_agent()
    
    while True:
        topic = get_user_topic()
        if not topic:
            break
            
        results = research_workflow(topic, research_executor)
        display_results(results)
        
        continue_research = input("\nWould you like to research another topic? (y/n): ").strip().lower()
        if continue_research != 'y':
            break
    
    print("\nThank you for using the AI Research Assistant!")

if __name__ == "__main__":
    main()


============= AI RESEARCH ASSISTANT ==============

I can help research any topic and generate a detailed report.
Just tell me what you'd like to research!



/home/istiak/anaconda3/lib/python3.12/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/tmp/ipykernel_6901/381316319.py:64: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return self.copy(update={**kwargs, **({"stop": stop} if stop else {})})



What would you like to research? (or 'quit' to exit):  exit


Please enter a more specific topic (at least 2 words).



What would you like to research? (or 'quit' to exit):  exit


Please enter a more specific topic (at least 2 words).



What would you like to research? (or 'quit' to exit):  quit



Thank you for using the AI Research Assistant!
